In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from torch.optim import lr_scheduler
import copy
import shutil  
import re

In [ ]:


# Applying Transforms to the Data
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=224),
        transforms.CenterCrop(size=224),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}



In [ ]:


# Load the Data

# Set train and valid directory paths

dataset = '/content/drive/MyDrive/Dataset/chest_xray'

train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'test')

# Batch size
bs = 32

# Number of classes
num_classes = len(os.listdir(valid_directory))
print(num_classes)

# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'val': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])
}

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['val'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data_loader = DataLoader(data['val'], batch_size=bs, shuffle=True)



3
{0: 'NORMAL', 1: 'PNEUMONIA'}


In [ ]:
train_data_size, valid_data_size

(5216, 624)

In [ ]:
# Load pretrained ResNet18 Model
alexnet = models.alexnet(pretrained=True)
alexnet = alexnet.to('cuda:0')

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [ ]:


# Freeze model parameters
for param in alexnet.parameters():
    param.requires_grad = False



In [ ]:
num_ftrs = alexnet.classifier[6].in_features

In [ ]:
for param in alexnet.classifier[6].parameters():
  param.requires_grad = True

alexnet = alexnet.to('cuda:0')

In [ ]:
# Define Optimizer and Loss Function
loss_func = nn.CrossEntropyLoss()
# optimizer = optim.Adam(resnet18.parameters())

In [ ]:


optimizer = torch.optim.SGD(filter(lambda x: x.requires_grad, alexnet.parameters()), lr=0.001, momentum=0.9)



In [ ]:
def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(train_data_loader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(valid_data_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/valid_data_size 
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, dataset+'_model_'+str(epoch)+'.pt')
            
    return model, history

In [ ]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train the model
num_epochs = 30
trained_model, history = train_and_validate(alexnet, loss_func, optimizer, num_epochs)

torch.save(history, dataset+'_history.pt')



Epoch: 1/30
Epoch : 000, Training: Loss: 0.5939, Accuracy: 84.7584%, 
		Validation : Loss : 0.5596, Accuracy: 82.0513%, Time: 1677.9376s
Epoch: 2/30
Epoch : 001, Training: Loss: 0.2693, Accuracy: 89.9156%, 
		Validation : Loss : 0.8336, Accuracy: 79.9679%, Time: 111.5680s
Epoch: 3/30
Epoch : 002, Training: Loss: 0.2994, Accuracy: 88.9571%, 
		Validation : Loss : 0.5124, Accuracy: 84.6154%, Time: 113.5092s
Epoch: 4/30
Epoch : 003, Training: Loss: 0.3672, Accuracy: 88.4778%, 
		Validation : Loss : 0.5485, Accuracy: 87.1795%, Time: 112.5868s
Epoch: 5/30
Epoch : 004, Training: Loss: 0.3197, Accuracy: 89.7239%, 
		Validation : Loss : 0.6562, Accuracy: 83.8141%, Time: 114.0131s
Epoch: 6/30
Epoch : 005, Training: Loss: 0.2871, Accuracy: 89.9923%, 
		Validation : Loss : 1.0930, Accuracy: 78.3654%, Time: 114.0113s
Epoch: 7/30
Epoch : 006, Training: Loss: 0.2964, Accuracy: 90.0115%, 
		Validation : Loss : 1.0473, Accuracy: 79.4872%, Time: 113.0141s
Epoch: 8/30
Epoch : 007, Training: Loss: 0.2596